# Ejercicio Módulo 5 - Dataset Swiss
**Inteligencia Artificial - CEIA - FIUBA**

**Damián Smilovich**

Para aprender sobre regresión, vamos a utilizar un dataset clásico llamado Swiss, que proviene originalmente del lenguaje R. Este dataset contiene datos socioeconómicos de 47 provincias suizas a fines del siglo XIX. Cada fila representa una provincia, y las variables reflejan características demográficas y sociales relevantes para ese contexto histórico.

## Variables

- `Location`: Provincia donde se midieron los datos.
- `Fertility`: Tasa de fertilidad (número promedio de hijos por mujer)
- `Agriculture`:` Porcentaje de hombres ocupados en agricultura
- `Examination`: Porcentaje de hombres que completaron exámenes de educación superior
- `Education`: Nivel promedio de educación (escala arbitraria)
- `Catholic`: Porcentaje de población católica
- `Infant.Mortality`: Tasa de mortalidad infantil (por cada 1000 nacidos vivos)

## Que queremos predecir?

Vamos a utilizar este dataset para predecir la tasa de fertilidad en cada provincia mediante diferentes métodos de regresión.

--- 

Siguiendo el procedimiento típico de Machine Learning, vamos a leer los datos y separarlos en los datasets de entrenamiento y testeo utilizando Scikit-Learn...

In [42]:
import pandas as pd

df = pd.read_csv("swiss.csv")

df.head()

,Location,Fertility,Agriculture,Examination,Education,Catholic,Infant.Mortality
0,Courtelary,80.2,17.0,15,12,9.96,22.2
1,Delemont,83.1,45.1,6,9,84.84,22.2
2,Franches-Mnt,92.5,39.7,5,5,93.40,20.2
3,Moutier,85.8,36.5,12,7,33.77,20.3
4,Neuveville,76.9,43.5,17,15,5.16,20.6


In [43]:
print(f"Tenemos {df.shape[0]} observaciones")

Tenemos 47 observaciones


Obtenemos la variable objetivo (`Fertility`) y, por otro lado, los atributos (quitamos `Location` ya que no es un atributo numérico relevante para la regresión)

In [44]:
X = df.drop(["Fertility", "Location"], axis=1)
y = df["Fertility"]

Dado que tenemos pocas observaciones, vamos a separar el dataset en un 50% para entrenamiento y 50% para testeo:

In [45]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)

## Regresión lineal múltiple

Arranquemos la primera parte del ejercicio. Para eso, vamos a entrenar un modelo de regresión lineal múltiple usando todos los atributos. Para ello debes:

1. Escalar los atributos usando `StandardScaler`
2. Entrenar el modelo usando el dataset de entrenamiento.
3. Obtener las predicciones sobre el dataset de testeo.
4. Calcular las métricas MAE, MSE  y $R^2$, e imprimir los resultados.

In [36]:
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (r2_score, mean_absolute_error, 
                             mean_squared_error, root_mean_squared_error, 
                             mean_absolute_percentage_error)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

model = LinearRegression()
model.fit(X_train_scaled, y_train)
y_completo = model.predict(X_test_scaled)

r2 = r2_score(y_test, y_completo)
mae = mean_absolute_error(y_test, y_completo)
rmse = root_mean_squared_error(y_test, y_completo)

print(f"R-cuadrado en test: {round(r2, 2)}")
print(f"Error absoluto medio: {round(mae, 2)}")
print(f"Raíz de error cuadrático medio: {round(rmse, 2)}")

# como para tener una idea de como compara con simplemente la media:
y_base = np.full_like(y_test, y_train.mean(), dtype=float)

r2_base = r2_score(y_test, y_base)
mae_base = mean_absolute_error(y_test, y_base)
rmse_base = root_mean_squared_error(y_test, y_base)

print(f"R-cuadrado en test, media: {round(r2_base, 2)}")
print(f"Error absoluto medio, media: {round(mae_base, 2)}")
print(f"Raíz de error cuadrático medio, media: {round(rmse_base, 2)}")

R-cuadrado en test: 0.57
Error absoluto medio: 6.09
Raíz de error cuadrático medio: 8.03
R-cuadrado en test, media: -0.0
Error absoluto medio, media: 8.77
Raíz de error cuadrático medio, media: 12.31


## Modelo con regularización

Para mejorar nuestro modelo, vamos a explorar técnicas de regresión lineal con regularización, que nos permiten controlar el sobreajuste y seleccionar variables relevantes automáticamente.

Existen dos variantes muy populares:

- Una penaliza la suma de los cuadrados de los coeficientes (regularización L2).
- La otra penaliza la suma del valor absoluto de los coeficientes (regularización L1).

Ambas ayudan a mejorar la generalización, pero una de ellas además puede eliminar variables (coeficientes exactamente cero), lo que ayuda a identificar qué atributos son realmente importantes.

Tu tarea:

1. Elegí correctamente cuál de los dos métodos de regularización usar para este problema. 
    - Pista: Queremos que el modelo sea capaz de hacer una selección automática de variables, dejando fuera aquellas que no aportan.
2. Implementá un pipeline que incluya escalado y el modelo elegido.
3. Buscá automáticamente el mejor valor del hiperparámetro de regularización (alpha) usando validación cruzada usando 3-folds.
4. Entrená el modelo con los datos de entrenamiento y obtené las predicciones para el set de testeo.
5. Calcular las métricas MAE, MSE  y $R^2$, e imprimir los resultados.
6. Imprimí los coeficientes resultantes e identificá qué variables fueron eliminadas (coeficiente = 0).

In [34]:
import numpy as np

from sklearn.linear_model import LassoCV, RidgeCV

Paremos un momento para entender qué hacen LassoCV y RidgeCV antes de continuar con la resolución:

> Tanto `LassoCV` como `RidgeCV` son implementaciones de regresión lineal con regularización que incluyen la búsqueda automática del mejor hiperparámetro alpha mediante validación cruzada.
>
> Ambos métodos prueban distintos valores de alpha y eligen el que minimiza el error del modelo, facilitando el proceso de ajuste sin necesidad de una búsqueda manual.
>
> Internamente, utilizan la métrica del error cuadrático medio (MSE) para evaluar el rendimiento del modelo en cada fold de la validación cruzada.
>
> Por ejemplo, si llamás a RidgeCV(alphas=alphas, cv=5), se hará una validación cruzada de 5 folds utilizando los valores de alpha que vos le pases, y se seleccionará el que obtenga el menor MSE promedio.
> 
> Una vez elegido el mejor alpha, el modelo final se entrena con todos los datos de entrenamiento usando ese valor.

¡Listo! Con todo lo que vimos hasta ahora, ya estás en condiciones de resolver esta parte y completar los 6 puntos propuestos

In [47]:
alphas = np.logspace(-4, 1, 500)

lasso_model = LassoCV(alphas=alphas, cv=3)
lasso_model.fit(X_train_scaled, y_train)

print(f"El alpha resultante es: {lasso_model.alpha_}")

coefs = lasso_model.coef_
for col, coef in zip(X, coefs):
    print(f"Peso para el feature {col}: {coef}")

y_lasso = lasso_model.predict(X_test_scaled)

r2_lasso = r2_score(y_test, y_lasso)
mae_lasso = mean_absolute_error(y_test, y_lasso)
rmse_lasso = root_mean_squared_error(y_test, y_lasso)

print(f"R-cuadrado en test, media: {round(r2_lasso, 2)}")
print(f"Error absoluto medio, media: {round(mae_lasso, 2)}")
print(f"Raíz de error cuadrático medio, media: {round(rmse_lasso, 2)}")

El alpha resultante es: 2.18110892419152
Peso para el feature Agriculture: -0.0
Peso para el feature Examination: -1.76243263029612
Peso para el feature Education: -2.4049047665643566
Peso para el feature Catholic: 1.7196487461209222
Peso para el feature Infant.Mortality: 3.2004065147188947
R-cuadrado en test, media: 0.58
Error absoluto medio, media: 6.0
Raíz de error cuadrático medio, media: 8.02


## Comparación de modelos y conclusiones

Completá la siguiente tabla con las métricas obtenidas para cada uno de los modelos que entrenaste:

| Modelo                        | MAE | MSE | $R^2$ |
| ----------------------------- | --- | --- | ----- |
| Regresión Lineal              |  6.09   |  8.03   |   0.57    |
| Modelo Regularizado (L1) |  6.0   |  8.02   |   0.58    |


> ⚠️ Asegurate de cambiar el nombre del modelo `Modelo Regularizado (L1 o L2)` según el modelo que usaste (Lasso o Ridge).

### Justificación

**¿Cuál de los modelos te parece que tuvo un mejor desempeño general?**

Tené en cuenta las tres métricas al responder, y también pensá en la complejidad del modelo (por ejemplo, si eliminó variables innecesarias).

Escribí tu respuesta a continuación:

El feature eliminado, Agricultura, no parece tener un peso fuerte en el resultado, el modelo no mejora (apreciablemente) las métricas pero resulta más sencillo. Así y todo, hacer validación cruzada y selección del hiperparámetro, para no obtener mejoras en las métricas, no parece tener sentido.